In [3]:
import os
import numpy as np
import pandas as pd
from joblib import load
from sklearn.preprocessing import StandardScaler

# 백테스트 데이터 로드
backtesting_path = 'feature_database_0801_final_ver.xlsx'
backtesting_data = pd.read_excel(backtesting_path, sheet_name=None)

# 인덱스를 날짜 형식으로 변환 후 문자열로 변경
for keyword, df in backtesting_data.items():
    df.reset_index(drop=True, inplace=True)
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    df.set_index('date', inplace=True)
    print(f"{keyword} 키워드의 백테스트 데이터:")
    print(df.head())

Sheet1 키워드의 백테스트 데이터:
              WTI  DGS2  DGS10  TIPS    VIX    PPI      코스피    per   pbr  \
date                                                                       
2010-01-01    NaN   NaN    NaN   NaN    NaN    NaN      NaN    NaN   NaN   
2010-01-02    NaN   NaN    NaN   NaN    NaN    NaN      NaN    NaN   NaN   
2010-01-03    NaN   NaN    NaN   NaN    NaN    NaN      NaN    NaN   NaN   
2010-01-04  81.52  1.09   3.85  1.47  20.04  170.8  1696.14  23.87  1.35   
2010-01-05  81.74  1.01   3.77  1.43  19.35  170.8  1690.62  23.80  1.35   

               원달러  ...    발틱운임     scfi  미 소맥 선물   미 대두 선물       항공 여객  \
date                ...                                                   
2010-01-01     NaN  ...     NaN  1072.58    540.75   1048.63  6210262.0   
2010-01-02     NaN  ...     NaN  1072.58       NaN       NaN  6210262.0   
2010-01-03     NaN  ...     NaN  1072.58       NaN       NaN  6210262.0   
2010-01-04  1154.8  ...  3140.0  1072.58    557.63   1058.13  6210262.

In [11]:
# 저장된 모델 로드
model_dir = "plk_folder_05"
loaded_models = {}
for keyword in backtesting_data.keys():
    model_path = os.path.join(model_dir, f"{keyword}_model.pkl")
    if os.path.exists(model_path):
        loaded_models[keyword] = load(model_path)
        print(f"{keyword} 키워드의 모델을 {model_path}에서 로드했습니다.")
    else:
        print(f"{keyword} 키워드의 모델 파일이 {model_path}에 존재하지 않습니다.")

auto 키워드의 모델을 plk_folder_05\auto_model.pkl에서 로드했습니다.
construct 키워드의 모델을 plk_folder_05\construct_model.pkl에서 로드했습니다.
capital_market 키워드의 모델을 plk_folder_05\capital_market_model.pkl에서 로드했습니다.
chemicals 키워드의 모델을 plk_folder_05\chemicals_model.pkl에서 로드했습니다.
equipment 키워드의 모델을 plk_folder_05\equipment_model.pkl에서 로드했습니다.
transport 키워드의 모델을 plk_folder_05\transport_model.pkl에서 로드했습니다.
semi 키워드의 모델을 plk_folder_05\semi_model.pkl에서 로드했습니다.
bank 키워드의 모델을 plk_folder_05\bank_model.pkl에서 로드했습니다.
steel 키워드의 모델을 plk_folder_05\steel_model.pkl에서 로드했습니다.
telecom 키워드의 모델을 plk_folder_05\telecom_model.pkl에서 로드했습니다.
staples 키워드의 모델을 plk_folder_05\staples_model.pkl에서 로드했습니다.
discretionary 키워드의 모델을 plk_folder_05\discretionary_model.pkl에서 로드했습니다.
kospi 키워드의 모델을 plk_folder_05\kospi_model.pkl에서 로드했습니다.


In [12]:
# 각 모델의 예측이 특정 패턴을 따르는 경우, 예측 결과가 동일하게 나타날 수 있습니다.
#예를 들어, 모델이 특정한 임계값을 기준으로 예측을 수행하는 경우, 입력 데이터가 그 임계값을 넘지 않는다면 결과는 동일할 수 있습니다.
# 즉 우리가 수행한 모델이 일정 값을 넘기면 S_buy buy sell이므로 오류가 생겼던 것 같ㅇ므
from sklearn.preprocessing import StandardScaler

# 각 행에 대해 예측 수행
all_predictions = {}

for keyword, df in backtesting_data.items():
    if keyword in loaded_models:
        model = loaded_models[keyword]
        predictions = []
        scaler = StandardScaler()
        df_scaled = scaler.fit_transform(df)
        for idx, row in enumerate(df_scaled):
            date = df.index[idx]  # 인덱스가 날짜 형식임
            X_test = row.reshape(1, -1)
            probabilities = model.predict_proba(X_test)[0]
            predictions.append({
                'date': date,
                'S_buy': probabilities[0],
                'buy': probabilities[1],
                'sell': probabilities[2]
            })
        all_predictions[keyword] = pd.DataFrame(predictions)
        print(f"\n{keyword} 키워드의 예측 결과:")
        print(all_predictions[keyword].head())


auto 키워드의 예측 결과:
         date     S_buy       buy      sell
0  2010-05-03  0.892379  0.107621  0.000000
1  2010-06-01  0.159710  0.370177  0.470114
2  2010-07-01  0.263158  0.473684  0.263158
3  2010-08-02  0.367211  0.053958  0.578831
4  2010-09-01  0.943837  0.053826  0.002338

construct 키워드의 예측 결과:
         date     S_buy       buy      sell
0  2010-05-03  0.098148  0.151852  0.750000
1  2010-06-01  0.300000  0.550000  0.150000
2  2010-07-01  0.300000  0.647686  0.052314
3  2010-08-02  0.250000  0.500000  0.250000
4  2010-09-01  0.996242  0.003033  0.000725

capital_market 키워드의 예측 결과:
         date     S_buy       buy      sell
0  2010-05-03  0.110974  0.116914  0.772112
1  2010-06-01  0.150000  0.750000  0.100000
2  2010-07-01  0.893035  0.106965  0.000000
3  2010-08-02  0.054427  0.155482  0.790091
4  2010-09-01  0.943569  0.055450  0.000980

chemicals 키워드의 예측 결과:
         date     S_buy       buy      sell
0  2010-05-03  0.052860  0.159352  0.787787
1  2010-06-01  0.947368  0.0

In [13]:
# 예측 결과를 엑셀 파일로 저장
output_path = "predictions_v5.xlsx"
with pd.ExcelWriter(output_path) as writer:
    for keyword, predictions_df in all_predictions.items():
        predictions_df.to_excel(writer, sheet_name=keyword, index=False)
print(f"예측 결과가 {output_path}에 저장되었습니다.")

예측 결과가 predictions_v5.xlsx에 저장되었습니다.


: 

In [4]:
import os
import numpy as np
import pandas as pd
from joblib import load
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

# 설정
CONFIG = {
    'backtesting_path': 'backtesting_raw_0702.xlsx',
    'model_dir': 'plk_folder_05_1',
    'output_path': 'predictions_v5_1.xlsx'
}

def load_and_preprocess_data(file_path):
    """백테스팅 데이터를 로드하고 전처리합니다."""
    try:
        data = pd.read_excel(file_path, sheet_name=None)
        for keyword, df in data.items():
            df.reset_index(drop=True, inplace=True)
            df['date'] = pd.to_datetime(df['date'], errors='coerce')
            df['date'] = df['date'].dt.strftime('%Y-%m-%d')
            df.set_index('date', inplace=True)
            print(f"{keyword} 키워드의 백테스트 데이터:")
            print(df.head())
        return data
    except Exception as e:
        print(f"데이터 로드 중 오류 발생: {e}")
        return None

def load_models(model_dir, keywords):
    """저장된 모델을 로드합니다."""
    loaded_models = {}
    for keyword in keywords:
        model_path = os.path.join(model_dir, f"{keyword}_model.pkl")
        try:
            if os.path.exists(model_path):
                loaded_models[keyword] = load(model_path)
                print(f"{keyword} 키워드의 모델을 {model_path}에서 로드했습니다.")
            else:
                print(f"{keyword} 키워드의 모델 파일이 {model_path}에 존재하지 않습니다.")
        except Exception as e:
            print(f"{keyword} 키워드의 모델 로드 중 오류 발생: {e}")
    return loaded_models

def predict_with_model(model, df):
    """모델을 사용하여 예측을 수행합니다."""
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df)
    predictions = []
    for idx, row in enumerate(df_scaled):
        date = df.index[idx]
        X_test = row.reshape(1, -1)
        probabilities = model.predict_proba(X_test)[0]
        predictions.append({
            'date': date,
            'S_buy': probabilities[0],
            'buy': probabilities[1],
            'sell': probabilities[2]
        })
    return pd.DataFrame(predictions)

def save_predictions(predictions, output_path):
    """예측 결과를 엑셀 파일로 저장합니다."""
    try:
        with pd.ExcelWriter(output_path) as writer:
            for keyword, predictions_df in predictions.items():
                predictions_df.to_excel(writer, sheet_name=keyword, index=False)
        print(f"예측 결과가 {output_path}에 저장되었습니다.")
    except Exception as e:
        print(f"결과 저장 중 오류 발생: {e}")

def main():
    # 데이터 로드 및 전처리
    backtesting_data = load_and_preprocess_data(CONFIG['backtesting_path'])
    if backtesting_data is None:
        return

    # 모델 로드
    loaded_models = load_models(CONFIG['model_dir'], backtesting_data.keys())

    # 예측 수행
    all_predictions = {}
    for keyword, df in tqdm(backtesting_data.items(), desc="예측 수행 중"):
        if keyword in loaded_models:
            model = loaded_models[keyword]
            predictions = predict_with_model(model, df)
            all_predictions[keyword] = predictions
            print(f"\n{keyword} 키워드의 예측 결과:")
            print(predictions.head())

    # 결과 저장
    save_predictions(all_predictions, CONFIG['output_path'])

if __name__ == "__main__":
    main()

auto 키워드의 백테스트 데이터:
              WTI  DGS2  DGS10  TIPS    VIX    PPI    per   pbr     원달러  \
date                                                                      
2010-05-03  86.19  1.00   3.72  1.32  20.19  173.9  15.23  1.26  1118.6   
2010-06-01  72.70  0.78   3.29  1.32  35.54  175.2  14.75  1.21  1216.5   
2010-07-01  72.95  0.63   2.96  1.21  32.86  176.1  15.06  1.25  1228.7   
2010-08-02  81.25  0.56   2.99  1.13  22.01  174.8  15.23  1.31  1172.5   
2010-09-01  73.97  0.50   2.58  1.02  23.89  174.7  15.13  1.29  1184.7   

                 원엔  ...   구리선물_60  중국고정자산투자(yoy%)_60  중국 PMI_60   금 선물_60  \
date                 ...                                                     
2010-05-03  1189.62  ...  0.002319          -0.002954  -0.000025  0.001183   
2010-06-01  1334.47  ... -0.002042          -0.000427   0.001456  0.001863   
2010-07-01  1391.03  ... -0.004316          -0.000458  -0.000573  0.001617   
2010-08-02  1354.71  ... -0.000225          -0.000569  -0.001476

예측 수행 중:   8%|▊         | 1/13 [00:02<00:30,  2.55s/it]


auto 키워드의 예측 결과:
         date     S_buy       buy      sell
0  2010-05-03  0.992424  0.007576  0.000000
1  2010-06-01  0.135632  0.400422  0.463946
2  2010-07-01  0.333333  0.334188  0.332479
3  2010-08-02  0.398467  0.068346  0.533186
4  2010-09-01  0.861215  0.135824  0.002961


예측 수행 중:  15%|█▌        | 2/13 [00:05<00:27,  2.52s/it]


construct 키워드의 예측 결과:
         date     S_buy       buy      sell
0  2010-05-03  0.066667  0.200000  0.733333
1  2010-06-01  0.333333  0.533333  0.133333
2  2010-07-01  0.400000  0.598582  0.001418
3  2010-08-02  0.266667  0.533333  0.200000
4  2010-09-01  0.998425  0.001575  0.000000


...

예측 수행 중:  92%|█████████▏| 12/13 [00:30<00:02,  2.56s/it]


discretionary 키워드의 예측 결과:
         date     S_buy       buy      sell
0  2010-05-03  0.250000  0.650926  0.099074
1  2010-06-01  0.498859  0.449648  0.051493
2  2010-07-01  0.103199  0.840639  0.056162
3  2010-08-02  0.000340  0.843549  0.156111
4  2010-09-01  0.250000  0.650000  0.100000


예측 수행 중: 100%|██████████| 13/13 [00:33<00:00,  2.54s/it]


kospi 키워드의 예측 결과:
         date  S_buy   buy  sell
0  2010-05-03   0.05  0.30  0.65
1  2010-06-01   0.20  0.80  0.00
2  2010-07-01   0.70  0.25  0.05
3  2010-08-02   0.00  0.10  0.90
4  2010-09-01   0.95  0.05  0.00


예측 결과가 predictions_v5_1.xlsx에 저장되었습니다.
